## Climate time series analysis
* Charge and preprocess data
* Time series clustering 
* Causal analysis

In [1]:
import pandas as pd
import os
import datetime

In [5]:
col_names = pd.read_csv('../dades/col_names.txt', header=None)
df = pd.DataFrame()
df.loc[:,'timestamp'] = pd.read_csv('../dades/data_per_pix/-0.5,-48.5.csv', header=None).loc[:,0].apply(datetime.datetime.fromtimestamp)

In [8]:
i = - 65
df[str('-3.5,'+ str(i+0.5))] = pd.read_csv('../dades/data_per_pix/' + str('-3.5,'+ str(i+0.5) +'.csv'), header=None).loc[:, (col_names == 'maxTXx_cum_T_ERA_2')[0]]

In [2]:
len(os.listdir('../dades/data_per_pix/'))

13072

In [20]:
df = pd.read_csv('../dades/data_per_pix/' + str('-3.5,'+ str(i+0.5) +'.csv'), header=None).iloc[:,4225]

In [11]:
df

,0,1,2,3,4,5,6,7,8,9,...,6785,6786,6787,6788,6789,6790,6791,6792,6793,6794
0,3.155300e+08,-3.5,-64.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,4.0,0,NaN,0.0,NaN,NaN,NaN,NaN
1,3.259900e+08,-3.5,-64.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,NaN,0.0,5,NaN,7.0,NaN,NaN,NaN,NaN
2,3.182100e+08,-3.5,-64.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,2.0,4,NaN,8.0,NaN,NaN,NaN,NaN
3,3.207200e+08,-3.5,-64.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,NaN,1.0,4,NaN,4.0,NaN,NaN,NaN,NaN
4,3.234000e+08,-3.5,-64.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,NaN,0.0,7,NaN,9.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,1.304200e+09,-3.5,-64.5,7.0,7.0,7.0,3.0,3.0,3.0,3.0,...,2,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
380,1.306900e+09,-3.5,-64.5,4.0,7.0,7.0,4.0,4.0,4.0,4.0,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
381,1.309500e+09,-3.5,-64.5,4.0,4.0,7.0,4.0,4.0,4.0,4.0,...,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
382,1.312200e+09,-3.5,-64.5,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import numpy as np

col_names[col_names[0].str.find('maxTXx_cum_T_ERA_2') != -1]

,0
4225,maxTXx_cum_T_ERA_2


In [21]:
variables_names = ['maxTXx_cum_T_ERA_2']
variables_ind = [4225]

0      300.05
1      300.33
2      301.03
3      301.03
4      299.93
        ...  
379    299.97
380    300.54
381    300.54
382    301.33
383    301.33
Name: 4225, Length: 384, dtype: float64